## Homework 2
1. Input image from eos_map_x.png.
2. Try to detect the location of the <b>player</b> (in cyan color), and the location of the nearest <b>shrine</b> (in red color). (hint: use <i>inRange()</i>)
3. Try to detect the facing direction of the <b>player</b>. (hint: possibly use <i>morphologyEx()</i>, or <i>findContours(), minEnclosingTriangle()</i> to determine the player axis)
4. Draw a yellow line indicating the facing direction of the <b>player</b>.
5. Draw a yellow line from the <b>player</b> to the <b>shrine</b>.
4. Compute the required <b>rotating angle</b> so the player is facing to the shrine. 
(positive angle means clockwise rotation, negative angle means counterclockwise rotaion) (hint: use <i>atan2()</i>)
5. Print the rotating angle on top-left corner of the output images. (in degree, not radian)
6. Write a simple report in a separate cell.
7. Upload your Jupyter code file (*.ipynb)

In [47]:
import cv2
import numpy as np

In [48]:
image = cv2.imread('eos_map_0.png')

if image is None:
    print("Error: Unable to load image.")
else:
    print("Image loaded successfully.")

Image loaded successfully.


In [49]:
# Define the range of red and blue in HSV
lower_red = np.array([0, 100, 100])
upper_red = np.array([0, 255, 255])
lower_blue = np.array([90, 100, 100])
upper_blue  = np.array([120, 255, 255])

# Convert the image to HSV
hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
red_mask = cv2.inRange(hsv, lower_red, upper_red)
blue_mask = cv2.inRange(hsv, lower_blue, upper_blue)


# Apply the mask to the image
red_masked_image = cv2.bitwise_and(image, image, mask=red_mask)
blue_masked_image = cv2.bitwise_and(image, image, mask=blue_mask)

cv2.imshow('HSV Picture', hsv)
cv2.imshow('Original Image', image)
cv2.imshow('Red Mask', red_mask)
cv2.imshow('Red Masked Image', red_masked_image)
cv2.imshow('Blue Mask', blue_mask)
cv2.imshow('Blue Masked Image', blue_masked_image)
cv2.waitKey()
cv2.destroyAllWindows()

In [50]:
kernel = np.ones((5,5), np.uint8)
red_mask = cv2.morphologyEx(red_mask, cv2.MORPH_CLOSE, kernel)
blue_mask = cv2.morphologyEx(blue_mask, cv2.MORPH_CLOSE, kernel)

In [51]:
red_contours, _ = cv2.findContours(red_mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
red_center = []
red_areas = []

threshold_area = 100

for contour in red_contours:

    area = cv2.contourArea(contour)

    if area > threshold_area:

        moments = cv2.moments(contour)
        if moments["m00"] != 0:
            cx = int(moments["m10"] / moments["m00"])
            cy = int(moments["m01"] / moments["m00"])
            red_center.append((cx, cy))  
            red_areas.append(area)       
max_red_area = max(red_areas)

for i, center in enumerate(red_center):
    print(f"Center point {i+1}: {center}, Area: {red_areas[i]}")

print("Max Red Area:", max_red_area)

Center point 1: (83, 201), Area: 123.0
Center point 2: (117, 86), Area: 129.5
Max Red Area: 129.5


In [52]:
blue_contours, _ = cv2.findContours(blue_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
blue_centers      = []  
blue_areas        = []

blue_threshold_area = 40

for contour in blue_contours:
    area = cv2.contourArea(contour)  # Calculate the area of the contour
    if area > blue_threshold_area:
        # If the area is greater than the threshold, calculate the center point
        moments = cv2.moments(contour)
        if moments["m00"] != 0:  # Ensure the area is not zero to avoid division by zero
            cx = int(moments["m10"] / moments["m00"])  # x-coordinate of centroid
            cy = int(moments["m01"] / moments["m00"])  # y-coordinate of centroid
            blue_centers.append((cx, cy))  # Append the center point to blue_center list
            blue_areas.append(area)       # Append the area to blue_areas list

# Find the largest blue area
max_blue_area = max(blue_areas)

# Print the center points and areas of blue regions
for i, center in enumerate(blue_centers):
    print(f"Center point {i+1}: {center}, Area: {blue_areas[i]}")

print("Max Blue Area:", max_blue_area)

Center point 1: (114, 115), Area: 41.0
Max Blue Area: 41.0


In [53]:
if red_center and blue_centers:
    
    min_dist = float('inf')
    closest_red  = None
    closest_blue = None
    
    for red_center in red_center:
        for blue_center in blue_centers:
            dist = np.linalg.norm(np.array(blue_center) - np.array(red_center)) # 计算向量长度
        
            if dist < min_dist:
                min_dist = dist
                closest_red  = red_center 
                closest_blue = blue_center 

    closest_red  = tuple(closest_red)
    closest_blue = tuple(closest_blue)

    img_with_line = cv2.line(image.copy(), closest_red, closest_blue, (0, 255, 255),2)

In [67]:
epsilon = 0.1 * cv2.arcLength(blue_contours[0], True)
approx = cv2.approxPolyDP(blue_contours[0], epsilon, True)

hull = cv2.convexHull(approx)

hull_points = [tuple(point[0]) for point in hull]

side_length_ab =  ((hull_points[0][0] - hull_points[1][0])** 2 + (hull_points[0][1] - hull_points[1][1])** 2 ) ** 0.5
side_length_ac =  ((hull_points[0][0] - hull_points[2][0])** 2 + (hull_points[0][1] - hull_points[2][1])** 2 ) ** 0.5
side_length_bc =  ((hull_points[1][0] - hull_points[2][0])** 2 + (hull_points[1][1] - hull_points[2][1])** 2 ) ** 0.5


max_side_length = max(side_length_ab + side_length_ac , side_length_bc + side_length_ac, side_length_ab + side_length_bc)

if max_side_length == side_length_ab + side_length_ac: #a
    intersection = hull_points[0]
    
if max_side_length == side_length_ab + side_length_bc: #b
    intersection = hull_points[1]
    
if max_side_length == side_length_ac + side_length_bc: #c
    intersection = hull_points[2]

centroid_to_intersection_vector = np.array([intersection[0] - cx, intersection[1] - cy])

extended_point = (int(cx + 3 * centroid_to_intersection_vector[0]), int(cy + 3 * centroid_to_intersection_vector[1]))

img_with_yellow_lines = cv2.line(image.copy(), (cx, cy), intersection, (0, 255, 255), 2)

vec_red = np.array(red_center) - np.array(blue_center)

img_with_yellow_lines = cv2.resize(img_with_yellow_lines, None, fx=1, fy=1)
cv2.imshow('Image Yellow Lines and Intersection Point', img_with_yellow_lines)

img_with_line = cv2.resize(img_with_line, None, fx=3, fy=3)
cv2.imshow('Image with Line', img_with_line)

cv2.waitKey(0)
cv2.destroyAllWindows()

## Report